In [106]:
import pandas as pd
import re
import spacy
import json
from random import randint
from spacy.lang.en.stop_words import STOP_WORDS
from typing import Any, Optional, NamedTuple
from string import punctuation
from datetime import date
from itertools import combinations_with_replacement
from collections import Counter, defaultdict
from calendar import month_name, month_abbr

In [5]:
nlp = spacy.load("en_core_web_lg")

In [18]:
d = pd.read_csv('data/transaction_texts.csv')

In [90]:
class Transaction:
    
    def __init__(self, account_category, account_name, description):
        
        self.account_category: Optional[str] = account_category
        self.account_name: Optional[str] = account_name
        self.description: Optional[str] = description
            
        self.docs = defaultdict()
        self.labels = defaultdict()
    
    def __str__(self):
        return f'{"acc.cat:":<10}{self.account_category}\n{"acc.name:":<10}{self.account_name}\n{"desc:":<10}{self.description}'
    
    def similarity(self, another_transaction):
        
        sim_acc_cat = self.docs['account_category'].similarity(another_transaction.docs['account_category'])
        sim_acc_name = self.docs['account_name'].similarity(another_transaction.docs['account_name'])
        sim_desc = self.docs['description'].similarity(another_transaction.docs['description'])
        
        print(f'similarity: acc.cat: {sim_acc_cat:.4f} acc.name: {sim_acc_name:.4f} description: {sim_desc:.4f}')
        
        return (sim_acc_cat, sim_acc_name, sim_desc)

In [220]:
class Parser:
    
    def __init__(self):
        
        self.abbrs = json.load(open('data/acdic.json'))
        self.syns = {'revenue': set('revenue unbilled earnings gain income incoming proceeds profit return yield unrealized'.split()),
                     'expense': set('expense charge expenditure obligation spending spend overhead surcharge cost'.split()),
                     'accrue': set('accrue accumulate amass collect gather aggregate hoard'.split()),
                     'consolidation': set('consolidation merger strengthening unification amalgamation'.split()),
                     'adjustment': set('adjustment adjust adj alteration modification readjustment'.split()),
                     'recurring': set('regular daily monthly weekly fortnightly yearly repeat routine recurring'.split()),
                     'defer': set('adjourn delay postpone suspend'.split()),
                     'subscription': set('subscription'.split()),
                     'payroll': set('payroll salary wages pay remuneration paycheck earning'.split()),
                     'liability': set('liability debt burden obligation owing uninvoiced'.split()),
                     'rent': set('rent lease rental'.split()),
                     'depreciation': set('depreciation devaluation markdown deflation'.split()),
                     'tax': set('tax duty levy toll tariff excise'.split()),
                     'maintenance': set('maintenance repair'.split()),
                     'entertainment': set('entertainment recreation party'.split()),
                     'month': set(map(lambda x: x.lower(), set(month_name[1:]) | set(month_abbr[1:]))),
                     'food': set('food meal dinner lunch restaurant cafe brunch breakfast catering wine beer drinks'.split()),
                     'reverse': set('reverse back return inverse converse'.split())}
        
        for k in self.syns:
            syns_upd = set()
            for w in nlp(' '.join(self.syns[k])):
                syns_upd.add(w.lemma_.lower())
            self.syns[k] = syns_upd
    
    def run(self, st):
        
        _labs = defaultdict(set) 
        _doc = None
        
        if (not isinstance(st, str)) or (not st.strip()):
            return (_doc, _labs)
        
        _labs['dates'] = self.find_dates(st)
        
        st = st.lower().translate({ord(sep): '' for sep in './'})
        
        st = st.translate(str.maketrans({_: ' ' for _ in punctuation}))
        
        # unfold abbreviations
        st = ' '.join([self.abbrs.get(w, w).lower().replace(',','') for w in st.split()])
        
        # remove numbers
        st = ' '.join([w for w in st.split() if w.isalpha()])
        
        # remove stopwords
        st = ' '.join([w for w in st.split() if w not in STOP_WORDS])
        
        # replace multiple consecutive white spaces with a single one
        st = re.sub(r'\s+', ' ', st).strip()
        
        if st:
            _doc = nlp(st)
        
            for what in self.syns:
                for w in _doc:
                    if w.lemma_ in self.syns[what]:
                        _labs['labels'].add(what)
            
        return (_doc, _labs)
    
    def find_dates(self, st):
        
        plausible = dict(day=set(range(1,32)), 
                         month=set(range(1,13)), 
                         year=set(range(19,23)) | set(range(2019, 2023)))
        
        _dates = set()
        
        # try to create a set of found dates using a single - as separator 
        try:
            _dates |= {re.sub(r'[-\/.]+','-', d) for d in re.findall(r'\b\d{1,2}[-\/.]+\d{1,2}[-\/.]+\d{4}\b|'
                                                                     r'\b\d{4}[-\/.]+\d{1,2}[-\/.]+\d{1,2}\b|'
                                                                     r'\b\d{1,2}[-\/.]+\d{1,2}[-\/.]+\d{1,2}\b', st)}
        except:
            pass
        
        try:
            _dates |= {d + '-' + str(list(month_abbr).index(m.title())) + '-' + '20' + y 
                                  for d, m, y in re.findall(r'\b(\d{1,2})(' + '|'.join([m for m in month_abbr[1:]]) + r')(\d{2}\b)', st, flags=re.IGNORECASE)}
        except:
            pass
        
        if not _dates:
            return None
        
        # dates will be gathered in this set if they are valid
        dates = []
        
        for _date in _dates:
            
            _date_parts = _date.split('-')
            
            position_cands = defaultdict(set)
            
            for i, p in enumerate(_date_parts):
                for q in 'year month day'.split():
                    if int(p) in plausible[q]:
                        position_cands[i].add(q)
            
            if set(position_cands) != set(range(len(_date_parts))):
                continue
            
            for p0 in position_cands[0]:
                for p1 in position_cands[1] - {p0}:
                    for p2 in position_cands[2] - {p0} - {p1}:
                        
                        date_as_dict = {p0: int(_date_parts[0]), p1: int(_date_parts[1]), p2: int(_date_parts[2])}
                        
                        # make sure year is presented as 20YY
                        for _ in date_as_dict:
                            if (_ == 'year') and (len(str(date_as_dict[_])) == 2):
                                date_as_dict.update({_: int('20'+str(date_as_dict[_]))})
                                
                        dates.append(date(**date_as_dict))
        
        

        return dates   

In [221]:
p = Parser()

In [223]:
p.find_dates(r'd wkjf f 23--12-2022 and then wqd 1FEb22')

[datetime.date(2022, 12, 23),
 datetime.date(2022, 2, 1),
 datetime.date(2022, 1, 2)]

In [224]:
p.run(' ewfefe 2141x !@@@!2 2021--12-4 a.B 3/1452-1-23m  1FEb24  1.1.22 rev accrued maintenance&repairs -- reversed')

(ewfefe ab revenue accrued maintenance repairs reversed,
 defaultdict(set,
             {'dates': [datetime.date(2022, 1, 1),
               datetime.date(2022, 1, 1),
               datetime.date(2021, 4, 12),
               datetime.date(2021, 12, 4)],
              'labels': {'accrue', 'maintenance', 'revenue', 'reverse'}}))

In [95]:
t1 = Transaction(**d.iloc[randint(0, len(d)-1)])
print(t1)
t1.docs['account_category'], t1.labels['account_category']= p.run(t1.account_category)
t1.docs['account_name'], t1.labels['account_name']= p.run(t1.account_name)
t1.docs['description'], t1.labels['description']= p.run(t1.description)

acc.cat:  Accrued Liabilities
acc.name: Accrued Bonuses
desc:     July 2019 Provider Bonus Accruals - C. Longly


In [96]:
t1.labels

defaultdict(None,
            {'account_category': defaultdict(set,
                         {'dates': [], 'labels': {'accrue', 'liability'}}),
             'account_name': defaultdict(set,
                         {'dates': [], 'labels': {'accrue'}}),
             'description': defaultdict(set,
                         {'dates': [], 'labels': {'month'}})})

In [97]:
t2 = Transaction(**d.iloc[randint(0, len(d)-1)])
t2.docs['account_category'], t2.labels['account_category']= p.run(t2.account_category)
t2.docs['account_name'], t2.labels['account_name']= p.run(t2.account_name)
t2.docs['description'], t2.labels['description']= p.run(t2.description)

In [98]:
print(t2)

acc.cat:  Facility Expense
acc.name: Office supplies
desc:     Consolidation


In [99]:
t1.similarity(t2)

similarity: acc.cat: 0.5077 acc.name: 0.1431 description: 0.3652


(0.5076760164070834, 0.14314831746158355, 0.36522501795194706)

In [86]:
t2.labels

defaultdict(None,
            {'account_category': defaultdict(set,
                         {'dates': [], 'labels': {'revenue'}}),
             'account_name': defaultdict(set, {'dates': []}),
             'description': defaultdict(set,
                         {'dates': [datetime.date(2019, 8, 13)]})})

In [89]:
d.sample(30)

,account_category,account_name,description
24448628,Deferred Revenue - Current,Deferred Revenue-Maintenance and support (Soft...,ACP ASC 606 Reversals - 01/31/2018
36021397,Payroll Expense,Benefits Expenses,Consolidation
2777962,Deferred Revenue - Current,DefRev - Unbilled - Subscription,Consolidation
6645671,Accounts Payable,A/P Trade,Accounts Payable
4181982,Unbilled AR,Unbilled AR-Software Subscription (On Premise),ACP ASC 605 Reversals - 04/30/2017
27055614,Revenue - Services,Revenue - Unbilled Services,8/15/2019 21:11
19086538,Payroll Taxes,Employer's FICA,21JAN08
15718243,Fringe,Other Employee Benefits,"LOPEZ-HERNANDEZ, MARIA FERNANDA"
44731200,Deferred Revenue - Current,DefRev - Billed - Subscription,Consolidation
10208242,Cash and Cash Equivalents,Due To/From,Due To Fund 90001


In [88]:
Counter([s for t in d['account_category'] for s in t.lower().split()]).most_common()

[('revenue', 17412096),
 ('-', 16762708),
 ('expense', 9860679),
 ('current', 9664866),
 ('cash', 9278384),
 ('deferred', 8607848),
 ('and', 7276361),
 ('accounts', 6341614),
 ('equivalents', 4639192),
 ('payable', 4633078),
 ('subscriptions', 3760273),
 ('receivable', 3131301),
 ('benefit', 2336165),
 ('sales', 2186850),
 ('liabilities', 1695325),
 ('other', 1587099),
 ('interest', 1493643),
 ('salary', 1484949),
 ('wage', 1484949),
 ('depreciation', 1459381),
 ('of', 1251220),
 ('accrued', 1176907),
 ('cost', 1175317),
 ('intercompany', 1003193),
 ('services', 926370),
 ('software', 921539),
 ('payroll', 835486),
 ('inventory', 671186),
 ('taxes', 662356),
 ('accumulated', 599879),
 ('travel', 567191),
 ('costs', 560620),
 ('indirect', 560601),
 ('allocated', 560599),
 ('expenses', 551390),
 ('supports', 536112),
 ('assets', 479483),
 ('fringe', 454547),
 ('goods', 408811),
 ('sold', 408811),
 ('prepaid', 399707),
 ('gain/loss', 392654),
 ('currency', 379152),
 ('maintenance', 321215